# ToDo:

## trancribe.py
- [ ] change transcriber to batch processing
- [ ] download data from s3 if not available, and delete afterwards
- [ ] save also timestamp

## generate_slurm.py
- [ ] create jobs based on the size of the data

In [ ]:
import os

import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

import time

In [ ]:
# cwd 
print(os.getcwd())

In [ ]:
audio_dir = "world/snapshot.20240606153519/audio_files"

# list mp4 files
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.mp4')]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
audio_batch = audio_files[:4]

In [ ]:
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3", torch_dtype=torch.float16)
model.to(device)

# rertieve 8 long audio sequences
raw_audio = [whisper.load_audio(audio_file) for audio_file in audio_batch]


In [ ]:
start_time = time.time()

# process input, make sure to pass `padding='longest'` and `return_attention_mask=True`
inputs = processor(raw_audio, return_tensors="pt", truncation=False, padding="longest", return_attention_mask=True, sampling_rate=16_000)
inputs = inputs.to("cuda", torch.float16)

# activate `temperature_fallback` and repetition detection filters and condition on prev text
result = model.generate(**inputs, condition_on_prev_tokens=False, temperature=(0.0, 0.2, 0.4, 0.6, 0.8, 1.0), logprob_threshold=-1.0, compression_ratio_threshold=1.35, return_timestamps=True)

decoded = processor.batch_decode(result, skip_special_tokens=True)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"{elapsed_time}")

print(decoded)

In [ ]:
start_time = time.time()

with torch.no_grad():
    transcription = pipe(os.path.join(audio_dir, mp4_files[0]))

print(f"Time taken: {time.time() - start_time:.2f} seconds")    

In [ ]:
mp4_files[0]

In [ ]:
csv_file = "world/snapshot.20240606153519/video_list_eng_title.csv"

In [ ]:
import pandas as pd

file_df = pd.read_csv(csv_file)

In [ ]:
file_df.head()

In [ ]:
# sample 100 and save
file_df.sample(100).to_csv("world/snapshot.20240606153519/sample_100.csv", index=False)

In [ ]:
from s3utils import get_list_of_files_s3, load_file_from_s3

import pandas as pd

In [ ]:
files, sizes = get_list_of_files_s3("world/snapshot.20240606153519/audio_files", return_size=True)

In [4]:
import numpy as np

random_sizes = np.random.randint(1,1000, 20).tolist()

In [7]:
random_fns=np.arange(20).tolist()
random_fns = [str(a) for a in random_fns]

In [9]:
sorted_files = sorted(zip(random_sizes, random_fns))

files = [f for s, f in sorted_files]
sizes = [s for s, f in sorted_files]

In [ ]:
speech_files = [f for _, f in sorted(zip(random_sizes, random_fns))]


In [ ]:
# make a pandas datefram and sort
filenames, sizes = files

file_df = pd.DataFrame({"filename": filenames, "size": sizes})
file_df = file_df.sort_values("size", ascending=True)

In [ ]:
file_df.head()

In [ ]:
# download the first 100 files
import os
for i, row in file_df.head(100).iterrows():
    filename = row["filename"]
    
    # check if dir not exists
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    
    print(f"Downloading {filename}")
    load_file_from_s3(filename)